In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import ipyparallel as ipp
print(ipp.version_info)
from IPython.display import display

nproc = 4
cluster = ipp.Cluster(engines='mpi', n=nproc, shutdown_atexit=False)
print(cluster)
client = cluster.start_and_connect_sync(activate=True)
view = client[:]
client.ids


(8, 6, 1)
<Cluster(cluster_id='1687066366-d1qn', profile='default')>
Starting 4 engines with <class 'ipyparallel.cluster.launcher.MPIEngineSetLauncher'>
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:05<00:00,  1.46s/engine]


[0, 1, 2, 3]

In [3]:
!ipcluster list

PROFILE          CLUSTER ID                       RUNNING ENGINES LAUNCHER
default          1687062289-99ox                  True          4 MPI
default          1687066366-d1qn                  True          4 MPI


In [4]:
%autopx

%autopx enabled


In [5]:
from mpi4py.MPI import COMM_WORLD


numkgrp = 2

wrld_size = COMM_WORLD.Get_size()
wrld_rank = COMM_WORLD.Get_rank()

kgrp_size = wrld_size // numkgrp
idxkgrp = wrld_rank // numkgrp
kgrp_rank = wrld_rank % numkgrp


kgrp_intracomm = COMM_WORLD.Split(idxkgrp, wrld_rank)
kgrp_rank = kgrp_intracomm.Get_rank()

print(f"{wrld_rank}/{wrld_size} -> {idxkgrp} {kgrp_rank}")

[stdout:1] 1/4 -> 0 1


[stdout:3] 3/4 -> 1 1


[stdout:2] 2/4 -> 1 0


[stdout:0] 0/4 -> 0 0


In [6]:
from mpi4py.MPI import COMM_NULL, COMM_SELF
print(kgrp_intracomm.Get_parent() == COMM_NULL)

[stdout:0] True


[stdout:1] True


[stdout:3] True


[stdout:2] True


In [10]:
from mpi4py.MPI import Group, Intracomm

l_idxrank = [0, 2, 3]

newcomm = COMM_WORLD.Create_group(
    COMM_WORLD.Get_group().Incl(
        l_idxrank
    )
)

print(f"{wrld_rank}/{wrld_size}: {newcomm == COMM_NULL}, {newcomm.Get_parent() == COMM_NULL}, {isinstance(newcomm, Intracomm)}")

[stdout:0] 0/4: False, True, True


[stdout:3] 3/4: False, True, True


[stdout:1] 1/4: True, True, True


[stdout:2] 2/4: False, True, True


In [15]:
from mpi4py.MPI import IDENT, SIMILAR, UNEQUAL
if wrld_rank == 0:
    grp_int = Group.Intersection(COMM_WORLD.Get_group(), newcomm.Get_group())
    out = Group.Compare(newcomm.Get_group(), grp_int)
    print(out == IDENT, out == SIMILAR, out == UNEQUAL)

[stdout:0] True False False


In [16]:
COMM_NULL.barrier()

[1:execute]
---------------------------------------------------------------------------
Exception                                 Traceback (most recent call last)
Cell In[12], line 1
----> 1 COMM_NULL.barrier()

File mpi4py/MPI/Comm.pyx:1560, in mpi4py.MPI.Comm.barrier()

File mpi4py/MPI/msgpickle.pxi:690, in mpi4py.MPI.PyMPI_barrier()

Exception: MPI_ERR_COMM: invalid communicator
[0:execute]
---------------------------------------------------------------------------
Exception                                 Traceback (most recent call last)
Cell In[12], line 1
----> 1 COMM_NULL.barrier()

File mpi4py/MPI/Comm.pyx:1560, in mpi4py.MPI.Comm.barrier()

File mpi4py/MPI/msgpickle.pxi:690, in mpi4py.MPI.PyMPI_barrier()

Exception: MPI_ERR_COMM: invalid communicator
[2:execute]
---------------------------------------------------------------------------
Exception                                 Traceback (most recent call last)
Cell In[12], line 1
----> 1 COMM_NULL.barrier()

File mpi4py/MPI

AlreadyDisplayedError: 4 errors

In [10]:
%%px
import ipyparallel as ipp

import numpy as np
from mpi4py import MPI
from mpi4py.MPI import IN_PLACE, SUM, Intracomm
from mpi4py_fft import DistArray

from qtmpy.mpi.comm import CommMod



class Test:

    shape = (10, 10)

    def __init__(self, val):
        self.buffer = np.empty(self.shape, dtype='i8')
        self.buffer[:] = val

        
check = None
with CommMod(MPI.COMM_WORLD) as comm:
    comm_size, comm_rank = comm.Get_size(), comm.Get_rank()

    arr = Test(comm_rank)
    print(comm_size)
    if comm_rank == 1:
        print(1/0)
    val_final = (comm_size - 1) * comm_size // 2
    comm.Allreduce_sum_inplace(arr.buffer)
    check = np.all(arr.buffer == val_final)

print(f"{comm_rank}/{comm_size}: {check}")

[stdout:0] 4


[stdout:2] 4


[stdout:3] 4


[stdout:1] 4


[1:execute]
---------------------------------------------------------------------------
ZeroDivisionError                         Traceback (most recent call last)
File ~/qtmpy/src/qtmpy/mpi/comm.py:68, in CommMod.__exit__(***failed resolving arguments***)
     67     if isinstance(exc_val, Exception):
---> 68         raise exc_val
     69 except Exception as exc_val:

Cell In[3], line 28
     27 if comm_rank == 1:
---> 28     print(1/0)
     29 val_final = (comm_size - 1) * comm_size // 2

ZeroDivisionError: division by zero

The above exception was the direct cause of the following exception:

Exception                                 Traceback (most recent call last)
Cell In[3], line 22
     18         self.buffer[:] = val
     21 check = None
---> 22 with CommMod(MPI.COMM_WORLD) as comm:
     23     comm_size, comm_rank = comm.Get_size(), comm.Get_rank()
     25     arr = Test(comm_rank)

File ~/qtmpy/src/qtmpy/mpi/comm.py:70, in CommMod.__exit__(***failed resolving arguments***)
 

%px:  25%|███████████████████████████████████████████████████████▊                                                                                                                                                                       | 1/4 [00:31<01:35, 31.92s/tasks]


Received Keyboard Interrupt. Sending signal SIGINT to engines...


In [11]:
!ipcluster list

PROFILE          CLUSTER ID                       RUNNING ENGINES LAUNCHER
default          1687061752-k8hg                  True          0 MPI
default          1687061963-e5sr                  True          0 MPI
default          1687062222-1rs4                  True          4 MPI


In [ ]:
!ipcluster list

In [ ]:
cluster.restart_engines_sync()

In [5]:
import ipyparallel as ipp

import numpy as np
from mpi4py import MPI
from mpi4py.MPI import IN_PLACE, SUM
from mpi4py_fft import DistArray

from qtmpy.mpi.comm import CommMod



class Test:

    shape = (10, 10)

    def __init__(self, val):
        self.buffer = np.empty(self.shape, dtype='i8')
        self.buffer[:] = val

        
check = None
with CommMod(MPI.COMM_WORLD) as comm:
    comm_size, comm_rank = comm.Get_size(), comm.Get_rank()

    arr = Test(comm_rank)
    print(comm_size)
    val_final = (comm_size - 1) * comm_size // 2
    comm.Allreduce_sum_inplace(arr.buffer)
    check = np.all(arr.buffer == val_final)

print(f"{comm_rank}/{comm_size}: {check}")

[stdout:2] WARNING: More than 1 'ipyparallel.Cluster' instances detected. Interrupts might not be signalled to the correct cluster.
Selecting cluster-id: <Cluster(cluster_id='1687061963-e5sr', profile='default', controller=<running>, engine_sets=['1687061964'])>
4
2/4: True


[stdout:0] WARNING: More than 1 'ipyparallel.Cluster' instances detected. Interrupts might not be signalled to the correct cluster.
Selecting cluster-id: <Cluster(cluster_id='1687061963-e5sr', profile='default', controller=<running>, engine_sets=['1687061964'])>
4
0/4: True


[stdout:1] WARNING: More than 1 'ipyparallel.Cluster' instances detected. Interrupts might not be signalled to the correct cluster.
Selecting cluster-id: <Cluster(cluster_id='1687061963-e5sr', profile='default', controller=<running>, engine_sets=['1687061964'])>
4
1/4: True


[stdout:3] WARNING: More than 1 'ipyparallel.Cluster' instances detected. Interrupts might not be signalled to the correct cluster.
Selecting cluster-id: <Cluster(cluster_id='1687061963-e5sr', profile='default', controller=<running>, engine_sets=['1687061964'])>
4
3/4: True


In [6]:
print(check)

[stdout:0] True


[stdout:1] True


[stdout:2] True


[stdout:3] True
